# Simple Vector Store

In [18]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "sk-"
openai.api_key = os.environ["OPENAI_API_KEY"]

#### Load documents, build the VectorStoreIndex

In [19]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
)
from IPython.display import Markdown, display

In [6]:
# load documents
documents = SimpleDirectoryReader(
    "/Users/krishna/Career/Python/pandas/data/emails"
).load_data()

In [7]:
index = VectorStoreIndex.from_documents(documents)

In [8]:
# save index to disk
index.set_index_id("vector_index")
index.storage_context.persist("./storage")

In [9]:
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="storage")
# load index
index = load_index_from_storage(storage_context, index_id="vector_index")

INFO:llama_index.indices.loading:Loading indices with ids: ['vector_index']
Loading indices with ids: ['vector_index']


In [11]:
from llama_index.prompts import PromptTemplate

text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\nUsing"
    " both the context information and also using your own knowledge, answer"
    " the question: {query_str}\nIf the context isn't helpful, you can also"
    " answer the question on your own.\n"
)
text_qa_template = PromptTemplate(text_qa_template_str)

refine_template_str = (
    "The original question is as follows: {query_str}\nWe have provided an"
    " existing answer: {existing_answer}\nWe have the opportunity to refine"
    " the existing answer (only if needed) with some more context"
    " below.\n------------\n{context_msg}\n------------\nUsing both the new"
    " context and your own knowledge, update or repeat the existing answer.\n"
)
refine_template = PromptTemplate(refine_template_str)

#### Query Index

In [12]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine(text_qa_template=text_qa_template, refine_template=refine_template,response_mode="tree_summarize")
response = query_engine.query("what is the name of the iconic movie star? and what he is famous for?")
display(Markdown(f"<b>{response}</b>"))

<b>The name of the iconic movie star is Martin. He is famous for his remarkable talent and captivating performances on the silver screen. He has achieved tremendous success and recognition in the film industry, with a string of critically acclaimed roles and box office hits. Martin is known for his ability to immerse himself in diverse characters and bring them to life, earning him numerous awards and accolades. He is also known for his charisma, professionalism, and his ability to connect with fans all around the world. Martin's outstanding contributions to the world of cinema have redefined the standards of performance and storytelling, leaving an enduring legacy in the entertainment industry.</b>

In [17]:
response

Response(response="The name of the iconic movie star is Martin. He is famous for his remarkable talent and captivating performances on the silver screen. He has achieved tremendous success and recognition in the film industry, with a string of critically acclaimed roles and box office hits. Martin is known for his ability to immerse himself in diverse characters and bring them to life, earning him numerous awards and accolades. He is also known for his charisma, professionalism, and his ability to connect with fans all around the world. Martin's outstanding contributions to the world of cinema have redefined the standards of performance and storytelling, leaving an enduring legacy in the entertainment industry.", source_nodes=[NodeWithScore(node=TextNode(id_='10049135-e009-47c9-9369-74c79c31f6fe', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='45ff0008-0fc3-4e0a-b960-a8

**Query Index with SVM/Linear Regression**

Use Karpathy's [SVM-based](https://twitter.com/karpathy/status/1647025230546886658?s=20) approach. Set query as positive example, all other datapoints as negative examples, and then fit a hyperplane.

In [ ]:
query_modes = [
    "svm",
    "linear_regression",
    "logistic_regression",
]
for query_mode in query_modes:
    # set Logging to DEBUG for more detailed outputs
    query_engine = index.as_query_engine(vector_store_query_mode=query_mode)
    response = query_engine.query("What did the author do growing up?")
    print(f"Query mode: {query_mode}")
    display(Markdown(f"<b>{response}</b>"))

Query mode: svm


<b>The author wrote short stories and also started programming on an IBM 1401 computer in 9th grade. They later got their own microcomputer, a TRS-80, and wrote simple games, a rocket prediction program, and a word processor.</b>

Query mode: linear_regression


<b>The author worked on writing and programming growing up. They wrote short stories and also started programming on an IBM 1401 computer in 9th grade using an early version of Fortran. Later, they got a microcomputer and wrote simple games, a rocket prediction program, and a word processor.</b>

Query mode: logistic_regression


<b>The author worked on writing and programming growing up. They wrote short stories and also started programming on an IBM 1401 computer in 9th grade using an early version of Fortran. Later, they got a microcomputer and wrote simple games, a rocket prediction program, and a word processor.</b>

In [ ]:
display(Markdown(f"<b>{response}</b>"))

<b>The author worked on writing and programming growing up. They wrote short stories and also started programming on an IBM 1401 computer in 9th grade using an early version of Fortran. Later, they got a microcomputer and wrote simple games, a rocket prediction program, and a word processor.</b>

In [ ]:
print(response.source_nodes[0].text)

Now all I had to do was learn Italian.

Only stranieri (foreigners) had to take this entrance exam. In retrospect it may well have been a way of excluding them, because there were so many stranieri attracted by the idea of studying art in Florence that the Italian students would otherwise have been outnumbered. I was in decent shape at painting and drawing from the RISD foundation that summer, but I still don't know how I managed to pass the written exam. I remember that I answered the essay question by writing about Cezanne, and that I cranked up the intellectual level as high as I could to make the most of my limited vocabulary. [2]

I'm only up to age 25 and already there are such conspicuous patterns. Here I was, yet again about to attend some august institution in the hopes of learning about some prestigious subject, and yet again about to be disappointed. The students and faculty in the painting department at the Accademia were the nicest people you could imagine, but they had lo

**Query Index with custom embedding string**

In [ ]:
from llama_index.indices.query.schema import QueryBundle

In [ ]:
query_bundle = QueryBundle(
    query_str="What did the author do growing up?",
    custom_embedding_strs=["The author grew up painting."],
)
query_engine = index.as_query_engine()
response = query_engine.query(query_bundle)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

<b>The context does not provide information about what the author did growing up.</b>

**Use maximum marginal relevance**

Instead of ranking vectors purely by similarity, adds diversity to the documents by penalizing documents similar to ones that have already been found based on <a href="https://www.cs.cmu.edu/~jgc/publication/The_Use_MMR_Diversity_Based_LTMIR_1998.pdf">MMR</a> . A lower mmr_treshold increases diversity.

In [ ]:
query_engine = index.as_query_engine(
    vector_store_query_mode="mmr", vector_store_kwargs={"mmr_threshold": 0.2}
)
response = query_engine.query("What did the author do growing up?")

#### Get Sources

In [ ]:
print(response.get_formatted_sources())

> Source (Doc id: fa51aa2a-af68-450f-bb00-786df71f2cdc): What I Worked On

February 2021

Before college the two main things I worked on, outside of schoo...

> Source (Doc id: 4636483a-a416-4971-804f-abfb80a44378): Now all I had to do was learn Italian.

Only stranieri (foreigners) had to take this entrance exa...


#### Query Index with Filters

We can also filter our queries using metadata

In [ ]:
from llama_index import Document

doc = Document(text="target", metadata={"tag": "target"})

index.insert(doc)

In [ ]:
from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters

filters = MetadataFilters(
    filters=[ExactMatchFilter(key="tag", value="target")]
)

retriever = index.as_retriever(
    similarity_top_k=20,
    filters=filters,
)

source_nodes = retriever.retrieve("What did the author do growing up?")

In [ ]:
# retrieves only our target node, even though we set the top k to 20
print(len(source_nodes))

1


In [ ]:
print(source_nodes[0].text)
print(source_nodes[0].metadata)

target
{'tag': 'target'}
